In [2]:
%load_ext autoreload
%autoreload 2

from pathlib import Path
import pandas as pd
from helpers.data_helper import dataPrepper
from string import punctuation
import nltk
from nltk.corpus import stopwords


nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/andre/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# human

## Static

In [52]:
gold = pd.read_csv('../data/heden_samples_en_v2.tsv', sep='\t', index_col=0)

In [53]:
data = pd.read_csv('../data/2_new_run/12072023_171117-context.tsv', sep='\t', index_col=0)
data.dropna(inplace=True)

In [54]:
data['genre'] = data.film.apply(lambda x : x.split('/')[4].lower())

In [55]:
gold['src'], gold['tgt'] = map(list, zip(*[p.split(' - ') for p in gold.pair.to_list()]))
gold = gold[['src', 'tgt', 'score', 'Genre', 'class']]

In [56]:
for genre_data, genre_gold in zip(data.genre.unique(), gold.Genre.unique()):
    gold.loc[gold.Genre == genre_gold, 'Genre'] = genre_data

In [57]:
gold = gold.rename({'Genre': 'genre'}, axis=1)

In [58]:

data['src_lower'] = [i.lower() for i in data.src.to_list()]
data['tgt_lower'] = [i.lower() for i in data.tgt.to_list()]

In [59]:
samples_df = pd.DataFrame(columns=data.columns)
for idx, i in gold.iterrows():
    cur_df = data[(data['src_lower'] == i.src) & (data['tgt_lower'] == i.tgt) & (data.genre == i.genre)].copy()
    cur_df['gold_idx'] = idx
    samples_df = pd.concat([samples_df, cur_df])


In [60]:
not_used = [i for i in range(125) if i not in samples_df.gold_idx.unique()]

In [61]:
data[data.src_lower == 'handsomely']

,src_sent,tgt_sent,aligns,src,tgt,cosine_w,cosine_sent,film,sent_idx,labels,genre,src_lower,tgt_lower
1168,and they 're prepared to pay handsomely for it .,Ze betalen goed .,1-0 2-1 5-1 6-2 8-2 9-3,handsomely,goed,0.771272,0.543484,../../data/2_new_run/Action/AC06_Indiana_Jones...,140,creative shift,action,handsomely,goed


In [62]:
# data[(data.src_lower == 'it') & (data.tgt_lower == 'het') & (data.genre == 'children_family')]
data[(data.src_lower == 'tequilas')]

,src_sent,tgt_sent,aligns,src,tgt,cosine_w,cosine_sent,film,sent_idx,labels,genre,src_lower,tgt_lower
313,"Let 's have two tequilas , please , señorita .","Twee tequila's graag , señorita .",1-1 3-0 4-1 7-3 8-4 9-5,tequilas,tequila's,0.645099,0.131959,../../data/2_new_run/Comedy/CO09_The_Wrong_Mis...,41,creative shift,comedy,tequilas,tequila's


In [63]:
for idx, i in data[data.genre == 'children_family'].iterrows():
    if 'it' in i.src_sent.lower():
        print(i.src_sent, i.tgt_sent)
        print(i.aligns)
        print(i.film)
        break


Okay , here we go . I think it 's all set . Volgens mij moet hij het zo doen .
2-2 4-6 6-1 7-2 8-4 12-7
../../data/2_new_run/Children_Family/CH01_Christmas_Chronicles/txt/christmaschronicles_en-nl_t.lfa


In [64]:
print(len(gold.loc[not_used]))
display(gold.loc[not_used])

15


,src,tgt,score,genre,class
16,roadblock,wegversperring,0.479,action,Reproduction
35,handsomely,goed,0.588,children_family,Creative Shift
44,it,kun,0.478,children_family,Creative Shift
46,he,niet,0.502,children_family,Creative Shift
51,see,maken,0.473,comedy,Creative Shift
59,tequilas,tequila,0.412,comedy,Reproduction
66,have,success,0.565,comedy,Creative Shift
81,and,naar,0.521,documentary,Creative Shift
84,where,overal,0.475,documentary,Creative Shift
85,might,hebt,0.479,documentary,Creative Shift


In [65]:
missing_samples = pd.DataFrame(columns=data.columns)
for genre, cnt in zip(gold.loc[not_used].genre.unique(), gold.loc[not_used].genre.value_counts(sort=False).to_list()):
    print(genre, cnt)
    new_samples = data[
        (data.genre == genre) 
        & (~data.src.isin([*punctuation])) 
        & (~data.src.isin(stopwords.words('english')))
        & (~data.src.isin(gold.src.to_list()))
        & (~data.src.isin(['I', 'It', 'And', 'No', '\'re']))
        # & (~data.src.isin(contractions)) 
        # & (~data.src.isin(pronouns)) 
        # & (~data.src.isin(articles))
    ].sample(random_state=20, n=cnt)
    new_samples['gold_idx'] = gold[(gold.index.isin(not_used)) & (gold.genre == genre)].index.to_list()
    missing_samples = pd.concat([missing_samples, new_samples])
    
display(missing_samples)

action 1
children_family 3
comedy 3
documentary 4
drama 4


,src_sent,tgt_sent,aligns,src,tgt,cosine_w,cosine_sent,film,sent_idx,labels,genre,src_lower,tgt_lower,gold_idx
3112,Your mocking is thus infantile . He 's not my ...,Je gedrag is kinderachtig . Hij is m'n vriendj...,0-0 1-1 2-2 4-3 5-4 6-5 7-6 7-7 8-9 9-7 10-8 1...,boyfriend,vriendje,0.236011,0.346131,../../data/2_new_run/Action/AC10_The_Old_Guard...,485,reproduction,action,boyfriend,vriendje,16.0
4542,Come on !,Kom op .,0-0 1-1 2-2,Come,Kom,0.146900,0.164914,../../data/2_new_run/Children_Family/CH07_Jing...,730,reproduction,children_family,come,kom,35.0
3551,"I mean , this is n't that bad .",Het valt wel mee .,3-0 4-1 7-3 8-4,bad,mee,0.877471,0.792354,../../data/2_new_run/Children_Family/CH03_Clou...,538,creative shift,children_family,bad,mee,44.0
2795,Where the Willoughby are you going ?,Waar de Willoughby gaan jullie naartoe ? Weg .,0-0 1-1 2-2 5-3 6-6,Where,Waar,0.095694,0.059625,../../data/2_new_run/Children_Family/CH10_The_...,398,reproduction,children_family,where,waar,46.0
4708,"Yeah , you 're good . It 's okay . That 's okay .",Niks aan de hand . Helemaal niks aan de hand .,5-4 6-5 8-3 9-4 12-9 13-10,okay,hand,0.851907,0.773393,../../data/2_new_run/Comedy/CO10_Work_it/txt/w...,602,creative shift,comedy,okay,hand,51.0
3910,"Yes , I , am actually","Ja , ik was net",0-0 1-1 2-2 4-3 5-4,Yes,Ja,0.137677,0.241135,../../data/2_new_run/Comedy/CO07_Operation_Chr...,452,reproduction,comedy,yes,ja,59.0
6507,to pick where I wanna go to school .,om te kiezen waar ik wil studeren .,0-0 1-2 2-3 3-4 4-5 5-5 7-6 8-7,school,studeren,0.418748,0.238738,../../data/2_new_run/Comedy/CO08_The_Kissing_B...,875,creative shift,comedy,school,studeren,66.0
2713,Sure . Sure .,Natuurlijk .,0-0 2-0 3-1,Sure,Natuurlijk,0.195274,0.105951,../../data/2_new_run/Documentary/DO10_Why_did_...,306,reproduction,documentary,sure,natuurlijk,81.0
3114,"Mind you , the the population of San Francisco...",De bevolking van San Francisco was maar,4-0 5-1 6-2 7-3 8-4 9-5 10-6,population,bevolking,0.056527,0.087978,../../data/2_new_run/Documentary/DO09_Social_D...,388,reproduction,documentary,population,bevolking,84.0
7736,we decided to join Sea Shepherd to get up clos...,gingen we mee met Sea Shepherd om deze frontli...,0-1 3-3 4-4 5-5 6-6 7-12 9-10 10-10 12-8 13-8 ...,front,frontlinie,0.065606,0.230616,../../data/2_new_run/Documentary/DO08_Seaspira...,835,reproduction,documentary,front,frontlinie,85.0


In [66]:
for loop, i in enumerate(samples_df.gold_idx.unique()):
    if loop == 0:
        new_gold = samples_df[samples_df.gold_idx == i].reset_index(names='samples_idx').iloc[[0]]
    else:
        new_gold = pd.concat([new_gold, samples_df[samples_df.gold_idx == i].reset_index(names='samples_idx').iloc[[0]]])
new_gold.set_index('samples_idx', inplace=True)
display(new_gold)

,src_sent,tgt_sent,aligns,src,tgt,cosine_w,cosine_sent,film,sent_idx,labels,genre,src_lower,tgt_lower,gold_idx
samples_idx,,,,,,,,,,,,,,
3,The Inquisition has finally delivered Spain to...,"Andalusië , Spanje , 1492 De Inquisitie heeft ...",0-5 1-6 2-7 4-9 5-8 6-10 7-11 8-12 9-13,delivered,overgedragen,0.457493,0.332573,../../data/2_new_run/Action/AC01_Assassin_s_Cr...,0,reproduction,action,delivered,overgedragen,0.0
12,Sultan Muhammad and his people still hold out ...,Sultan Mohammed en zijn mensen zitten nog in G...,0-0 1-1 2-2 3-3 4-4 5-6 6-5 8-7 9-8 10-9,his,zijn,0.118527,0.063485,../../data/2_new_run/Action/AC01_Assassin_s_Cr...,1,reproduction,action,his,zijn,1.0
37,swear to honor our Order in the fight for free...,zweert gij trouw aan de Orde ?,0-0 1-0 2-2 3-4 4-5 6-4 10-6,swear,zweert,0.474395,0.534822,../../data/2_new_run/Action/AC01_Assassin_s_Cr...,5,creative shift,action,swear,zweert,2.0
55,I swear .,Dat zweer ik .,0-0 0-2 1-1 2-3,I,Dat,0.316410,0.203802,../../data/2_new_run/Action/AC01_Assassin_s_Cr...,8,creative shift,action,i,dat,3.0
203,Sit down . You 're making me nervous .,Ga zitten . U maakt me nerveus .,0-0 1-1 2-2 3-3 5-4 6-5 7-6 8-7,Sit,Ga,0.159237,0.134274,../../data/2_new_run/Action/AC01_Assassin_s_Cr...,37,reproduction,action,sit,ga,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2028,Poor men fighting a rich man 's war .,Armen die vochten voor de rijken .,0-0 1-0 2-1 2-3 3-4 4-5 7-5 8-6,Poor,Armen,0.301984,0.352154,../../data/2_new_run/Drama/DR02_News_of_the_Wo...,331,reproduction,drama,poor,armen,120.0
2065,"Point is , us old soldiers gotta live , right ?",Wij oud-soldaten moeten ook rondkomen .,3-0 4-1 5-1 6-2 7-4 10-5,gotta,moeten,0.612505,0.335632,../../data/2_new_run/Drama/DR02_News_of_the_Wo...,335,creative shift,drama,gotta,moeten,121.0
2348,"No , no . No , this is worthless .",Hier heb je niks aan .,6-0 7-1 8-3 9-5,worthless,niks,0.879349,0.784428,../../data/2_new_run/Drama/DR02_News_of_the_Wo...,389,creative shift,drama,worthless,niks,122.0


In [67]:
new_gold = pd.concat([new_gold, missing_samples])

In [68]:
new_gold.sort_values(by=['gold_idx'], inplace=True)

In [69]:
new_gold.reset_index(names='samples_idx', inplace=True)

In [70]:
new_gold.set_index('gold_idx', inplace=True)

In [71]:
new_gold.index = new_gold.index.astype('int')

In [72]:
new_gold.drop(columns=['src_lower', 'tgt_lower'], inplace=True)

In [73]:
new_gold['heden_labels'] = [i['class'] if idx not in not_used else float('nan') for idx, i in gold.iterrows()]

In [74]:
[[idx, i] for idx, i in enumerate(new_gold.columns)]

[[0, 'samples_idx'],
 [1, 'src_sent'],
 [2, 'tgt_sent'],
 [3, 'aligns'],
 [4, 'src'],
 [5, 'tgt'],
 [6, 'cosine_w'],
 [7, 'cosine_sent'],
 [8, 'film'],
 [9, 'sent_idx'],
 [10, 'labels'],
 [11, 'genre'],
 [12, 'heden_labels']]

In [75]:
test = [new_gold.columns.to_list()[i] for i in [0, 11, 8, 9, 4, 5, 1, 2, 3, 6, 7, 10]]

In [76]:
# new_gold[['src_sent', 'tgt_sent', 'src', 'tgt', 'labels', 'heden_labels']].to_csv('../data/2_new_run/annotation_data.tsv', sep='\t')
gold_changes = pd.read_csv('../data/2_new_run/annotation_data.tsv', sep='\t')

In [77]:
gold_changes = pd.concat([new_gold, gold_changes[['label_static', 'label_context']]], axis=1)

In [83]:
static_gold = gold_changes[['film', 'sent_idx', 'src', 'tgt', 'label_static']].copy()
static_gold.film = static_gold.film.apply(lambda x : x.split('/')[5].split('_')[0])
static_gold.to_csv('../data/static_gold.tsv', sep='\t')
static_gold

,film,sent_idx,src,tgt,label_static
0,AC01,0,delivered,overgedragen,Reproduction
1,AC01,1,his,zijn,Reproduction
2,AC01,5,swear,zweert,Reproduction
3,AC01,8,I,Dat,Creative Shift
4,AC01,37,Sit,Ga,Creative Shift
...,...,...,...,...,...
120,DR02,331,Poor,Armen,Reproduction
121,DR02,335,gotta,moeten,Reproduction
122,DR02,389,worthless,niks,Creative Shift
123,DR01,979,share,deel,Reproduction


In [89]:
static_gold.to_csv('../data/2_new_run/static_gold.tsv', sep='\t')

## Contextual

In [86]:
context_gold = gold_changes[['film', 'sent_idx', 'src_sent', 'tgt_sent', 'src', 'tgt', 'cosine_w', 'cosine_sent', 'label_context']].copy()
context_gold.film = context_gold.film.apply(lambda x : x.split('/')[5].split('_')[0])
context_gold.to_csv('../data/context_gold.tsv', sep='\t')
context_gold

,film,sent_idx,src_sent,tgt_sent,src,tgt,cosine_w,cosine_sent,label_context
0,AC01,0,The Inquisition has finally delivered Spain to...,"Andalusië , Spanje , 1492 De Inquisitie heeft ...",delivered,overgedragen,0.457493,0.332573,Reproduction
1,AC01,1,Sultan Muhammad and his people still hold out ...,Sultan Mohammed en zijn mensen zitten nog in G...,his,zijn,0.118527,0.063485,Reproduction
2,AC01,5,swear to honor our Order in the fight for free...,zweert gij trouw aan de Orde ?,swear,zweert,0.474395,0.534822,Reproduction
3,AC01,8,I swear .,Dat zweer ik .,I,Dat,0.316410,0.203802,Creative Shift
4,AC01,37,Sit down . You 're making me nervous .,Ga zitten . U maakt me nerveus .,Sit,Ga,0.159237,0.134274,Creative Shift
...,...,...,...,...,...,...,...,...,...
120,DR02,331,Poor men fighting a rich man 's war .,Armen die vochten voor de rijken .,Poor,Armen,0.301984,0.352154,Reproduction
121,DR02,335,"Point is , us old soldiers gotta live , right ?",Wij oud-soldaten moeten ook rondkomen .,gotta,moeten,0.612505,0.335632,Reproduction
122,DR02,389,"No , no . No , this is worthless .",Hier heb je niks aan .,worthless,niks,0.879349,0.784428,Creative Shift
123,DR01,979,I gave Bubba 's mama Bubba 's share .,gaf ik Bubba's deel aan zijn mama .,share,deel,0.129550,0.157482,Reproduction


In [91]:
context_gold.to_csv('../data/2_new_run/context_gold.tsv', sep='\t')

# Machine data

## Contextual

In [3]:
cont_gold = pd.read_csv('../data/context_gold.tsv', sep='\t', index_col=0)
cont_mt = pd.read_csv('../data/2_new_run/16072023_180924-context_mt.tsv', sep='\t', index_col=0)

In [4]:
cont_mt['file'] = cont_mt.film
cont_mt['genre'] = cont_mt.film.apply(lambda x : x.split('/')[4])
cont_mt['film'] = cont_mt.film.apply(lambda x : x.split('/')[5].split('_')[0])

In [5]:
cont_mt[:1]

,src_sent,tgt_sent,aligns,src,tgt,cosine_w,cosine_sent,film,sent_idx,labels,file,genre
0,The Inquisition has finally delivered Spain to...,De inquisitie heeft Spanje eindelijk uitgeleve...,0-0 1-1 2-2 3-4 4-5 5-3 6-6 7-7 8-8 9-9,The,De,0.098573,0.062565,AC01,0,reproduction,../../data/2_new_run/Action/AC01_Assassin_s_Cr...,Action


In [6]:
cont_gold[:1]

,film,sent_idx,src_sent,tgt_sent,src,tgt,cosine_w,cosine_sent,label_context
0,AC01,0,The Inquisition has finally delivered Spain to...,"Andalusië , Spanje , 1492 De Inquisitie heeft ...",delivered,overgedragen,0.457493,0.332573,Reproduction


In [7]:
cont_mt[(cont_mt.film == 'AC05') & (cont_mt.sent_idx == 126)]

,src_sent,tgt_sent,aligns,src,tgt,cosine_w,cosine_sent,film,sent_idx,labels,file,genre
794,"Dr. Jones , you promised","Dr. Jones , u beloofde",0-0 1-1 2-2 3-3 4-4,Dr.,Dr.,0.006006,0.034953,AC05,126,reproduction,../../data/2_new_run/Action/AC05_Indiana_Jones...,Action
795,"Dr. Jones , you promised","Dr. Jones , u beloofde",0-0 1-1 2-2 3-3 4-4,Jones,Jones,0.016279,0.034953,AC05,126,reproduction,../../data/2_new_run/Action/AC05_Indiana_Jones...,Action
796,"Dr. Jones , you promised","Dr. Jones , u beloofde",0-0 1-1 2-2 3-3 4-4,",",",",0.005020,0.034953,AC05,126,reproduction,../../data/2_new_run/Action/AC05_Indiana_Jones...,Action
797,"Dr. Jones , you promised","Dr. Jones , u beloofde",0-0 1-1 2-2 3-3 4-4,you,u,0.053091,0.034953,AC05,126,reproduction,../../data/2_new_run/Action/AC05_Indiana_Jones...,Action
798,"Dr. Jones , you promised","Dr. Jones , u beloofde",0-0 1-1 2-2 3-3 4-4,promised,beloofde,0.136501,0.034953,AC05,126,creative shift,../../data/2_new_run/Action/AC05_Indiana_Jones...,Action


In [8]:
cont_mt[(cont_mt.film == 'AC01') & (cont_mt.sent_idx == 0)]

,src_sent,tgt_sent,aligns,src,tgt,cosine_w,cosine_sent,film,sent_idx,labels,file,genre
0,The Inquisition has finally delivered Spain to...,De inquisitie heeft Spanje eindelijk uitgeleve...,0-0 1-1 2-2 3-4 4-5 5-3 6-6 7-7 8-8 9-9,The,De,0.098573,0.062565,AC01,0,reproduction,../../data/2_new_run/Action/AC01_Assassin_s_Cr...,Action
1,The Inquisition has finally delivered Spain to...,De inquisitie heeft Spanje eindelijk uitgeleve...,0-0 1-1 2-2 3-4 4-5 5-3 6-6 7-7 8-8 9-9,Inquisition,inquisitie,0.072921,0.062565,AC01,0,reproduction,../../data/2_new_run/Action/AC01_Assassin_s_Cr...,Action
2,The Inquisition has finally delivered Spain to...,De inquisitie heeft Spanje eindelijk uitgeleve...,0-0 1-1 2-2 3-4 4-5 5-3 6-6 7-7 8-8 9-9,has,heeft,0.122000,0.062565,AC01,0,reproduction,../../data/2_new_run/Action/AC01_Assassin_s_Cr...,Action
3,The Inquisition has finally delivered Spain to...,De inquisitie heeft Spanje eindelijk uitgeleve...,0-0 1-1 2-2 3-4 4-5 5-3 6-6 7-7 8-8 9-9,finally,eindelijk,0.064406,0.062565,AC01,0,reproduction,../../data/2_new_run/Action/AC01_Assassin_s_Cr...,Action
4,The Inquisition has finally delivered Spain to...,De inquisitie heeft Spanje eindelijk uitgeleve...,0-0 1-1 2-2 3-4 4-5 5-3 6-6 7-7 8-8 9-9,delivered,uitgeleverd,0.251047,0.062565,AC01,0,creative shift,../../data/2_new_run/Action/AC01_Assassin_s_Cr...,Action
5,The Inquisition has finally delivered Spain to...,De inquisitie heeft Spanje eindelijk uitgeleve...,0-0 1-1 2-2 3-4 4-5 5-3 6-6 7-7 8-8 9-9,Spain,Spanje,0.024725,0.062565,AC01,0,reproduction,../../data/2_new_run/Action/AC01_Assassin_s_Cr...,Action
6,The Inquisition has finally delivered Spain to...,De inquisitie heeft Spanje eindelijk uitgeleve...,0-0 1-1 2-2 3-4 4-5 5-3 6-6 7-7 8-8 9-9,to,aan,0.141253,0.062565,AC01,0,reproduction,../../data/2_new_run/Action/AC01_Assassin_s_Cr...,Action
7,The Inquisition has finally delivered Spain to...,De inquisitie heeft Spanje eindelijk uitgeleve...,0-0 1-1 2-2 3-4 4-5 5-3 6-6 7-7 8-8 9-9,the,de,0.173857,0.062565,AC01,0,creative shift,../../data/2_new_run/Action/AC01_Assassin_s_Cr...,Action
8,The Inquisition has finally delivered Spain to...,De inquisitie heeft Spanje eindelijk uitgeleve...,0-0 1-1 2-2 3-4 4-5 5-3 6-6 7-7 8-8 9-9,Templars,tempeliers,0.279878,0.062565,AC01,0,creative shift,../../data/2_new_run/Action/AC01_Assassin_s_Cr...,Action
9,The Inquisition has finally delivered Spain to...,De inquisitie heeft Spanje eindelijk uitgeleve...,0-0 1-1 2-2 3-4 4-5 5-3 6-6 7-7 8-8 9-9,.,.,0.098969,0.062565,AC01,0,reproduction,../../data/2_new_run/Action/AC01_Assassin_s_Cr...,Action


In [9]:
cont_gold[cont_gold.film == 'AC01']

,film,sent_idx,src_sent,tgt_sent,src,tgt,cosine_w,cosine_sent,label_context
0,AC01,0,The Inquisition has finally delivered Spain to...,"Andalusië , Spanje , 1492 De Inquisitie heeft ...",delivered,overgedragen,0.457493,0.332573,Reproduction
1,AC01,1,Sultan Muhammad and his people still hold out ...,Sultan Mohammed en zijn mensen zitten nog in G...,his,zijn,0.118527,0.063485,Reproduction
2,AC01,5,swear to honor our Order in the fight for free...,zweert gij trouw aan de Orde ?,swear,zweert,0.474395,0.534822,Reproduction
3,AC01,8,I swear .,Dat zweer ik .,I,Dat,0.316410,0.203802,Creative Shift
4,AC01,37,Sit down . You 're making me nervous .,Ga zitten . U maakt me nerveus .,Sit,Ga,0.159237,0.134274,Creative Shift
5,AC01,268,Who are you ?,Wie ben jij ?,are,ben,0.100883,0.073212,Creative Shift
12,AC01,443,You 're gonna kill the Creed !,Je vermoordt het Credo.,kill,vermoordt,0.448816,0.389783,Reproduction
20,AC01,313,The Lord spake,En de Heer sprak .,Lord,Heer,0.244897,0.487075,Reproduction


In [10]:
def cont_mt_creator(idx, cont_mt, row, cont_mt_new, pattern='full'):
    if pattern == 'full':
        cur_df_mt = cont_mt[(cont_mt.film == row.film) & (cont_mt.src == row.src) 
                            & (cont_mt.tgt == row.tgt) & (cont_mt.src_sent == row.src_sent)].copy()[:1]
        cur_df_mt['label_context'] = row.label_context
    elif pattern == 'src_ws':
        cur_df_mt = cont_mt[(cont_mt.film == row.film) & (cont_mt.src == row.src) & (cont_mt.src_sent == row.src_sent)].copy()[:1]
        # cur_df_mt = pd.concat([cur_df_mt, row], axis=1)
        cur_df_mt['label_context'] = float('nan')
    elif pattern == 'src_w':
        cur_df_mt = cont_mt[(cont_mt.film == row.film) & (cont_mt.src == row.src)].copy()[:1]
        cur_df_mt['label_context'] = float('nan')

    if cur_df_mt.empty:
        raise Exception('The dataframe should not be empty')

    cur_df_mt['pattern'] = pattern
    if idx == 0:
        cont_mt_new = cur_df_mt
    else:
        cont_mt_new = pd.concat([cont_mt_new, cur_df_mt])
    
    return cont_mt_new


cont_mt_new = []
for idx, row in cont_gold.iterrows():
    try:
        cont_mt_new = cont_mt_creator(idx, cont_mt, row, cont_mt_new)
    except Exception:
        try:
            cont_mt_new = cont_mt_creator(idx, cont_mt, row, cont_mt_new, pattern='src_ws')
        except Exception:
            try:
                cont_mt_new = cont_mt_creator(idx, cont_mt, row, cont_mt_new, pattern='src_w')
            except Exception:
                print(idx, row.film, row.sent_idx, row.src)
                cont_mt_new = cont_mt_new.reset_index(drop=True)
                cont_mt_new.loc[idx] = float('nan')
                row_info = {'film': row.film, 'src': row.src, 'tgt': row.tgt, 
                            'src_sent': row.src_sent, 'tgt_sent': row.tgt_sent, 
                            'sent_idx': row.sent_idx}
                cont_mt_new.loc[idx, row_info.keys()] = row_info.values()

cont_mt_new.reset_index(drop=True, inplace=True)
cont_mt_new['label_old'] = cont_gold.label_context 
cont_mt_new = cont_mt_new[['film', 'sent_idx', 'src_sent', 'tgt_sent', 'src', 'tgt', 'cosine_w', 'cosine_sent', 'label_old', 'pattern', 'label_context']]
cont_mt_new.sent_idx = cont_mt_new.sent_idx.map(lambda x : int(x))
display(cont_mt_new)

24 AC05 127 moment


,film,sent_idx,src_sent,tgt_sent,src,tgt,cosine_w,cosine_sent,label_old,pattern,label_context
0,AC01,0,The Inquisition has finally delivered Spain to...,De inquisitie heeft Spanje eindelijk uitgeleve...,delivered,uitgeleverd,0.251047,0.062565,Reproduction,src_ws,NaN
1,AC01,1,Sultan Muhammad and his people still hold out ...,Sultan Mohammed en zijn volk houden nog stand ...,his,zijn,0.095121,0.050959,Reproduction,full,Reproduction
2,AC01,5,swear to honor our Order in the fight for free...,om onze Orde te eren in de strijd voor vrijheid ?,swear,om,0.659257,0.110100,Reproduction,src_ws,NaN
3,AC01,8,I swear .,Dat zweer ik .,I,Dat,0.316410,0.203802,Creative Shift,full,Creative Shift
4,AC01,38,Sit down . You 're making me nervous .,Ga zitten . Je maakt me nerveus .,Sit,Ga,0.151804,0.128368,Creative Shift,full,Creative Shift
...,...,...,...,...,...,...,...,...,...,...,...
120,DR02,391,Poor men fighting a rich man 's war .,Arme mannen vochten een rijke oorlog .,Poor,Arme,0.176537,0.073245,Reproduction,src_ws,NaN
121,DR02,395,"Point is , us old soldiers gotta live , right ?",Wij oude soldaten moeten leven .,gotta,moeten,0.215512,0.198202,Reproduction,full,Reproduction
122,DR02,461,"No , no . No , this is worthless .","Nee , nee . Nee , dit is waardeloos .",worthless,waardeloos,0.099917,0.065806,Creative Shift,src_ws,NaN
123,DR01,1303,I gave Bubba 's mama Bubba 's share .,gaf ik Bubba's moeder Bubba's deel .,share,deel,0.498643,0.075324,Reproduction,full,Reproduction


In [11]:
cont_mt_new[cont_mt_new.pattern.isnull()]

,film,sent_idx,src_sent,tgt_sent,src,tgt,cosine_w,cosine_sent,label_old,pattern,label_context
24,AC05,127,"DONOVAN : Be along in a moment , dear .","Ik kom eraan , lieve .",moment,eraan,NaN,NaN,Creative Shift,NaN,NaN


In [12]:
mt_moment_repl = cont_mt[(cont_mt.film == 'AC05') & (cont_mt.cosine_w > 0.6)].sample(10, random_state=30)[7:8].copy()
display(mt_moment_repl)

,src_sent,tgt_sent,aligns,src,tgt,cosine_w,cosine_sent,film,sent_idx,labels,file,genre
3950,You dolt !,Idioot !,0-0 1-0 2-1,You,Idioot,0.785906,0.61409,AC05,566,creative shift,../../data/2_new_run/Action/AC05_Indiana_Jones...,Action


In [13]:
mt_moment_repl.iloc[0].src_sent

'You dolt !'

In [14]:
row_inp = {main_final[0].src_sent,
    'tgt_sent': mt_moment_repl.iloc[0].tgt_sent,
    'src': mt_moment_repl.iloc[0].src,
    'tgt': mt_moment_repl.iloc[0].tgt,
    'cosine_w': mt_moment_repl.iloc[0].cosine_w,
    'cosine_sent': mt_moment_repl.iloc[0].cosine_sent,
    'label_old': 'None',
    'pattern': 'None',
}
cont_mt_new.loc[24, row_inp.keys()] = row_inp.values()

In [15]:
# cont_mt_new.to_csv('../data/context_mt_gold.tsv', sep='\t')

## Syntax